# Cleaning the Data
This code pulls the csv (good_merged_data.csv) that was created in the Data Gathering file (1_movie_df.ipynb) and removes incomplete and unecessary information, corrects formatting, calculates the Return on Investment (ROI), and filters out some data that does not meet the desired criteria

In [1]:
# Set up Dependencies
from progress.bar import IncrementalBar
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import nltk
import json
import csv
import sys
import os
from scipy.stats import linregress
from pprint import pprint

In [2]:
# read the data back into the dataframe from the csv file
input_data_file = os.path.join('data_files','good_merged_data.csv')

# write the DataFrame data to the csv file
movie_df = pd.read_csv(input_data_file)

movie_df

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue,ROI%
0,Vulgar,2000,R,26 Apr 2002,87,"Crime, Drama, Thriller",English,NaN,5.3,3876,tt0120467,movie,Lions Gate Films,120000,14904,-87.58
1,Out of the Furnace,2013,R,06 Dec 2013,116,"Action, Crime, Drama, Thriller",English,2 wins & 9 nominations.,6.8,103339,tt1206543,movie,Relativity Media,22000000,15400000,-30.00
2,Coyote Ugly,2000,PG-13,04 Aug 2000,100,"Comedy, Drama, Music, Romance",English,4 wins & 8 nominations.,5.7,104644,tt0200550,movie,Buena Vista,45000000,113916474,153.15
3,Parental Guidance,2012,PG,25 Dec 2012,105,"Comedy, Family",English,1 win & 4 nominations.,6.0,24453,tt1047540,movie,20th Century Fox,25000000,119772232,379.09
4,Raising Helen,2004,PG-13,28 May 2004,119,"Comedy, Drama, Romance",English,1 nomination.,6.0,34868,tt0350028,movie,Buena Vista Pictures,50000000,49718611,-0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,Life,2017,R,24 Mar 2017,104,"Horror, Sci-Fi, Thriller","English, Japanese, Vietnamese",7 nominations.,6.6,199775,tt5442430,movie,Skydance Films,58000000,100541806,73.35
582,Filth,2013,R,24 Apr 2014,97,"Comedy, Crime, Drama","English, German, Scots",9 wins & 14 nominations.,7.1,98589,tt1450321,movie,Magnet Releasing,5000000,9114264,82.29
583,Masterminds,2016,PG-13,30 Sep 2016,95,"Biography, Comedy, Crime","English, Spanish",1 nomination.,5.8,48784,tt2461150,movie,Relativity Media,25000000,29200000,16.80
584,Planet 51,2009,PG,20 Nov 2009,91,"Animation, Adventure, Comedy, Family, Sci-Fi","English, Spanish",4 wins & 4 nominations.,6.0,49599,tt0762125,movie,Sony Pictures/TriStar Pictures,70000000,104945765,49.92


In [ ]:
# change the "unrated" value to be consistent
movie_df = movie_df.replace({'Not Rated':'Unrated','UNRATED':'Unrated'})

In [ ]:
# check to make sure there are no other divergent values & all the "unrated" values now match
movie_df['Rated'].value_counts()

In [ ]:
# check to see if all values in the Runtime column are in minutes
movie_df['Runtime'].value_counts()

In [ ]:
# remove the text from the Runtime column
movie_df['Runtime'] = movie_df.Runtime.str.replace(' min' , '')
movie_df

In [ ]:
# convert the Runtime column into numbers
movie_df['Runtime'] = movie_df['Runtime'].astype('float64')

In [ ]:
# create a new Dataframe with movies at least 60 minutes long
movie_runtime_df = movie_df.loc[movie_df['Runtime'] >= 60]
movie_runtime_df

In [ ]:
# change Runtime value into integers instead of decimals
movie_runtime_df['Runtime'] = movie_runtime_df['Runtime'].astype('int64')

In [ ]:
movie_runtime_df

In [ ]:
# create a new dataframe to include movies with a budget over $10,000
movie_clean_df = movie_runtime_df.loc[movie_runtime_df['Budget'] > 10000]
movie_clean_df

In [ ]:
# remove the unnecessary columns from the dataframe
movie_clean_df.drop(columns=['BoxOffice','Metascore','IMDB ID'], inplace=True)
movie_clean_df

In [ ]:
# find the records in the dataframe that do not have an entry for the Production company
movie_clean_df[movie_clean_df['Production'].isnull()==True]

In [ ]:
# fill in the empty Production companies with a value of unknown
movie_clean_df['Production'].fillna(value='Unknown', inplace=True)

In [ ]:
# check to make sure there are no more Production companies with no values
len(movie_clean_df[movie_clean_df['Production'].isna()==True])

In [ ]:
# calculate the Return on Investment (ROI) for each row and store it in a new column
movie_clean_df['ROI%'] = round(((movie_clean_df['Revenue'] - movie_clean_df['Budget']) / movie_clean_df['Budget']) * 100,2)
movie_clean_df

In [ ]:
# remove the commas from the number of votes values
movie_clean_df['imdbVotes'] = movie_clean_df['imdbVotes'].str.replace(',','',-1)

# change the votes value from strings to numbers
movie_clean_df['imdbVotes'] = movie_clean_df['imdbVotes'].astype('int64')
movie_clean_df

In [ ]:
# store the cleaned data in a csv file
output_data_file = os.path.join('data_files','good_merged_data.csv')

# write the DataFrame data to the csv file
movie_clean_df.to_csv(output_data_file, index=False)

In [3]:
# check for movies with an ROI over 5000% and check to make sure the outliers are valid entries
find = movie_df.loc[movie_df['ROI%'] > 5000]
find

,Title,Year,Rated,Released,Runtime,Genre,Language,Awards,imdbRating,imdbVotes,imdbID,Type,Production,Budget,Revenue,ROI%
273,Get Out,2017,R,24 Feb 2017,104,"Horror, Mystery, Thriller",English,Won 1 Oscar. Another 152 wins & 201 nominations.,7.7,463872,tt5052448,movie,Universal Pictures,4500000,255407969,5575.73
389,Catfish,2010,PG-13,01 Oct 2010,87,"Documentary, Drama, Mystery, Thriller",English,2 wins & 7 nominations.,7.1,39522,tt1584016,movie,Universal Pictures/Rogue,30000,3045943,10053.14
408,The Full Monty,1997,R,19 Sep 1997,91,"Comedy, Drama",English,Won 1 Oscar. Another 36 wins & 34 nominations.,7.2,96582,tt0119164,movie,Twentieth Century Fox Home Entertainment,3500000,257850122,7267.15


In [10]:
# get a list of all the Production companies
production_co = movie_df['Production'].value_counts()
print(production_co.index)


Index(['Warner Bros. Pictures', 'Universal Pictures', '20th Century Fox',
       'Paramount Pictures', 'Warner Home Video', 'Sony Pictures',
       'Buena Vista', 'New Line Cinema', 'Unknown', 'Warner Bros.',
       ...
       'MGM/Columbia', 'Cinema Libre', 'Touchstone Home Entertainment',
       'Wetnwildradio Films', 'Wellspring Media Inc.', 'Gatlin Pictures',
       'Universal Classics', 'Blue Collar Releasing',
       'Millenium Entertainment', 'Pure Flix Entertainment'],
      dtype='object', length=164)
